In [ ]:
# auditoria_precios.ipynb

# =======================
# AUDITORÍA DE PRECIOS
# =======================
# Requiere: pandas, matplotlib, seaborn, numpy

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# -----------------------------
# Función principal del módulo
# -----------------------------
def auditoria_precios(df, imprimir=True, umbral_alerta=200, ventana_movil=2):
    """
    Realiza una auditoría de precios por producto en un DataFrame con columnas:
    'id_pedido', 'nombre_producto', 'precio_producto', y opcionalmente 'fecha_pedido'.

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame con información de pedidos.
    imprimir : bool
        Si True, imprime el reporte. Si False, solo retorna los objetos.
    umbral_alerta : float
        Precio a partir del cual se consideran alertas de precio alto.
    ventana_movil : int
        Tamaño de la ventana para calcular el promedio móvil.

    Retorna:
    --------
    dict con:
        - precios_por_producto
        - productos_con_precios_variados
        - variaciones_precio
        - precio_mas_frecuente
        - alertas_precio
        - historico_precio
        - promedio_movil_precio
        - productos_inestables
        - clasificacion_precios
    """

    historico_precio = pd.DataFrame()
    promedio_movil_precio = pd.DataFrame()
    productos_inestables = pd.Series()
    clasificacion_precios = pd.Series()
    df = df.copy()

    if 'fecha_pedido' in df.columns:
        df['fecha_pedido'] = pd.to_datetime(df['fecha_pedido'])
        historico_precio = (
            df.groupby(['fecha_pedido', 'nombre_producto'])['precio_producto']
            .mean()
            .reset_index()
        )

        promedio_movil_precio = historico_precio.copy()
        promedio_movil_precio['promedio_movil'] = (
            promedio_movil_precio
            .sort_values('fecha_pedido')
            .groupby('nombre_producto')['precio_producto']
            .transform(lambda x: x.rolling(window=ventana_movil, min_periods=1).mean())
        )

        # Clasificación por coeficiente de variación del precio
        coef_var = (
            historico_precio.groupby('nombre_producto')['precio_producto']
            .agg(lambda x: np.std(x) / np.mean(x) if np.mean(x) != 0 else 0)
        )
        productos_inestables = coef_var[coef_var > 0.1]  # arbitrario: CV > 10%

    precios_por_producto = df.groupby('nombre_producto')['precio_producto'].unique()
    conteo_precios = df.groupby('nombre_producto')['precio_producto'].nunique()
    productos_con_precios_variados = conteo_precios[conteo_precios > 1]

    variaciones_precio = (
        df[['id_pedido', 'nombre_producto', 'precio_producto']]
        .drop_duplicates()
        .sort_values(['nombre_producto', 'precio_producto'])
        .reset_index(drop=True)
    )

    precio_mas_frecuente = df.groupby('nombre_producto')['precio_producto']\
                             .agg(lambda x: x.value_counts().idxmax())

    alertas_precio = df[df['precio_producto'] > umbral_alerta]

    # Clasificación de productos en económicos, medios y premium
    precios_medios = df.groupby('nombre_producto')['precio_producto'].mean()
    clasificacion_precios = pd.qcut(precios_medios, q=3, labels=['económico', 'medio', 'premium'])

    if imprimir:
        print("\U0001F50D AUDITORÍA DE PRECIOS POR PRODUCTO\n")

        print("1⃣  Precios distintos por producto:\n")
        for producto, precios in precios_por_producto.items():
            print(f"   - {producto}: {list(precios)}")

        print("\n" + "-"*50)
        print("2⃣  Productos con múltiples precios:\n")
        if not productos_con_precios_variados.empty:
            for producto, n in productos_con_precios_variados.items():
                print(f"   ⚠️ {producto} aparece con {n} precios distintos.")
        else:
            print("   ✅ No se detectaron productos con múltiples precios.")

        print("\n" + "-"*50)
        print("3⃣  Variaciones de precio por producto y pedido:\n")
        print(variaciones_precio.to_string(index=False))

        print("\n" + "-"*50)
        print("4⃣  Precio más frecuente por producto:\n")
        for producto, precio in precio_mas_frecuente.items():
            print(f"   - {producto}: {precio}")

        print("\n" + "-"*50)
        print(f"5⃣  Alertas de precio (>{umbral_alerta}):\n")
        if not alertas_precio.empty:
            print(alertas_precio[['id_pedido', 'nombre_producto', 'precio_producto']].to_string(index=False))
        else:
            print("   ✅ No hay precios que excedan el umbral de alerta.")

        if not historico_precio.empty:
            print("\n" + "-"*50)
            print("6⃣  Evolución temporal del precio promedio por producto:\n")
            print(historico_precio.to_string(index=False))

            print("\n" + "-"*50)
            print(f"7⃣  Promedio móvil ({ventana_movil} fechas):\n")
            print(promedio_movil_precio.to_string(index=False))

            print("\n" + "-"*50)
            print("8⃣  Productos inestables (coeficiente de variación > 10%):\n")
            if not productos_inestables.empty:
                for prod, cv in productos_inestables.items():
                    print(f"   ⚠️ {prod} (CV={cv:.2%})")
            else:
                print("   ✅ Todos los productos tienen precios estables en el tiempo.")

        print("\n" + "-"*50)
        print("9⃣  Clasificación de productos por nivel de precio:\n")
        for prod, clasif in clasificacion_precios.items():
            print(f"   - {prod}: {clasif}")

        print("\n✅ Auditoría completada.")

    return {
        'precios_por_producto': precios_por_producto,
        'productos_con_precios_variados': productos_con_precios_variados,
        'variaciones_precio': variaciones_precio,
        'precio_mas_frecuente': precio_mas_frecuente,
        'alertas_precio': alertas_precio,
        'historico_precio': historico_precio,
        'promedio_movil_precio': promedio_movil_precio,
        'productos_inestables': productos_inestables,
        'clasificacion_precios': clasificacion_precios
    }

# -----------------------------
# Ejemplo de uso con datos demo
# -----------------------------
data = {
    'id_pedido': [1, 1, 2, 2, 3, 3, 4],
    'nombre_producto': ['Mouse', 'Teclado', 'Mouse', 'Teclado', 'Mouse', 'Monitor', 'Monitor'],
    'precio_producto': [10, 20, 12, 20, 10, 250, 270],
    'fecha_pedido': ["2024-01-01", "2024-01-01", "2024-01-05", "2024-01-05", "2024-02-10", "2024-02-15", "2024-03-01"]
}

# Crear DataFrame de ejemplo
df = pd.DataFrame(data)

# Ejecutar auditoría
resultados = auditoria_precios(df)

# Exportar resultados
resultados['variaciones_precio'].to_csv("variaciones_precios.csv", index=False)
resultados['variaciones_precio'].to_excel("variaciones_precios.xlsx", index=False)
resultados['alertas_precio'].to_csv("alertas_precios.csv", index=False)
resultados['historico_precio'].to_csv("historico_precios.csv", index=False)
resultados['promedio_movil_precio'].to_csv("promedio_movil_precio.csv", index=False)

# Visualización
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='nombre_producto', y='precio_producto', palette='Set2')
plt.title('Distribución de precios por producto')
plt.grid(True)
plt.tight_layout()
plt.show()

if not resultados['historico_precio'].empty:
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=resultados['historico_precio'], x='fecha_pedido', y='precio_producto', hue='nombre_producto', marker="o")
    plt.title('Evolución temporal del precio promedio por producto')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 6))
    sns.lineplot(data=resultados['promedio_movil_precio'], x='fecha_pedido', y='promedio_movil', hue='nombre_producto', marker="o")
    plt.title('Promedio móvil del precio por producto')
    plt.grid(True)
    plt.tight_layout()
    plt.show()


🔍 AUDITORÍA DE PRECIOS POR PRODUCTO

1⃣  Precios distintos por producto:

   - Monitor: [np.int64(250), np.int64(270)]
   - Mouse: [np.int64(10), np.int64(12)]
   - Teclado: [np.int64(20)]

--------------------------------------------------
2⃣  Productos con múltiples precios:

   ⚠️ Monitor aparece con 2 precios distintos.
   ⚠️ Mouse aparece con 2 precios distintos.

--------------------------------------------------
3⃣  Variaciones de precio por producto y pedido:

 id_pedido nombre_producto  precio_producto
         3         Monitor              250
         4         Monitor              270
         1           Mouse               10
         3           Mouse               10
         2           Mouse               12
         1         Teclado               20
         2         Teclado               20

--------------------------------------------------
4⃣  Precio más frecuente por producto:

   - Monitor: 250
   - Mouse: 10
   - Teclado: 20

✅ Auditoría completada.
